# szia-ai-tools: Core Functionality
This notebook demonstrates how to use the core functions and classes when developing components and applications. 

In [ ]:
# Auto reload imports
%reload_ext autoreload
%autoreload 2

# Visualization
%matplotlib inline

# Imports
import pandas as pd
import os

The example locations include file paths used to demonstrate how to process locally stored documents. You can add your own PDF, TXT, MD, or HTML files to try it out.

In [ ]:
example_urls = [
    "https://www.lynxanalytics.com/generative-ai-platform",
    "https://en.wikipedia.org/wiki/GPT-3",
    "https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)",
    "https://en.wikipedia.org/wiki/Generative_pre-trained_transformer",
    "https://en.wikipedia.org/wiki/Large_language_model",
    "https://arxiv.org/pdf/1706.03762",
]

example_locations = [
    '../data/example_doc_to_parse/ChatGPT - Wikipedia.html', 
    '../README.md', 
    '../data/example_doc_to_parse/wiki_GPT2.txt', 
    '../data/example_doc_to_parse/2402.06196.pdf'
]

## The DocumentLibrary Class

The `DocumentLibrary` is a utility for ingesting and processing documents such as PDF, TXT, MD, and HTML files. It can be initialized with a list of `Document` instances or directly via local file paths and/or URLs. When URLs are provided, the library attempts to crawl and retrieve their content (with more advanced crawling planned in the future). All documents are parsed and converted into standardized Markdown format before being chunked. Two chunking strategies are available: `simple`, which splits text by token count, and `simple_intervals`, which respects structural markers (e.g., headings, paragraphs) defined in `text_split_order`, aiming for clean, context-aware chunks within specified size limits. Metadata—such as source and error status—is tracked throughout processing, and sources can be manually enabled or disabled. The processed chunks can be exported as a `pandas.DataFrame` or a Python list (with metadata), making the `DocumentLibrary` ready for integration with downstream RAG pipelines.

In [ ]:
from sziaaitls.core.document_library import DocumentLibrary

mydoclib = await DocumentLibrary.create(
    urls=example_urls, # can be empty
    file_paths=example_locations, # can be empty
    chunk_method="simple_intervals",
    max_chunk_size=768,
    min_chunk_size=256,
    min_leftover_chunk_size=128,
    ignore_single_newline=True,
)

In [ ]:
mydoclib.documents[0].metadata

Instances of the `Document` class contain an internal variable called page_content. While each Document can parse and chunk itself, the `DocumentLibrary` is designed to handle these operations collectively.

In [ ]:
first_document = mydoclib.documents[0]
print(first_document.metadata)  # print metadata of the first document

print(type(first_document))  # there are MarkdownDocument, HtmlDocument, PdfDocument, TextDocument classes
print(first_document.page_content[:20]) 

parsed_doc_content = await first_document.get_parsed_content()
print(parsed_doc_content[:100]) 

The `get_chunks_with_metadata` function returns all chunks along with their associated metadata. Each Document instance retains its parsed and chunked content after the initial processing step.

In [ ]:
_chs = await mydoclib.get_chunks_with_metadata()
_chs[0]

In [ ]:
# the pandas DataFrame export
pdf_chunks = await mydoclib.get_chunk_df()
pdf_chunks.head(2)


The `DocumentLibrary`'s metadata stores key information about each contained document at the URI level

In [ ]:
mydoclib.get_metadata_df()

We can visualize a histogram of the chunk sizes to verify whether they fall within the specified minimum and maximum token limits:

In [ ]:
pdf_chunks.chunk_token_size.plot(kind='hist', bins=50, title='PDF Chunk Token Size Distribution')

## The TextEmbedder Class

The `TextEmbedder` is a utility class that converts chunks of text into vector embeddings, which are useful for semantic search, clustering, or RAG (Retrieval-Augmented Generation) pipelines. It supports multiple backends - for example, `"openai"` for hosted models or `"ollama"` for local inference.

In [ ]:
from sziaaitls.core.embedders import TextEmbedder
openai_embedder = TextEmbedder("openai", api_key=os.environ.get("OPENAI_API_KEY"), model="text-embedding-3-small", rate_limit=3000, period=60, max_batch_tokens=32768)
pdf_chunks['content_embedding'] = await openai_embedder.acreate(
    pdf_chunks['chunk_content'].tolist())
pdf_chunks.head(2)

To use the Ollama embedder, ensure that Ollama is installed and running (on Linux/Mac, run `ollama serve` in the terminal). Also, make sure you have downloaded the desired model by running `ollama pull nomic-embed-text`.

In [ ]:
ollama_embedder = TextEmbedder("ollama", model="nomic-embed-text")
pdf_chunks['content_embedding'] = await ollama_embedder.acreate(
    pdf_chunks['chunk_content'].tolist())
pdf_chunks.head(2)

### The LLMPromptProcessor Class

The `LLMPromptProcessor` class provides a unified interface for sending prompts to language models, supporting both OpenAI and Ollama-compatible models. It is designed for flexibility and ease of integration across different backends.

It accepts a `ChatCompletionPrompt`—which is a structured list of `Message` objects (each representing a role: `system`, `user`, or `assistant`)—and returns a single `Message` as the model's response.

In [ ]:
from sziaaitls.core.llms import LLMPromptProcessor, Message, ChatCompletionPrompt
prompt_text = [
    {'role':'system', 'content': 'You are a robot that tells a joke about the topic the user mentions.'}, 
    {'role':'user', 'content': 'Tell me a joke about cats.'},
]
llm = LLMPromptProcessor(
    api_key=os.environ.get("OPENAI_API_KEY"),
    model="gpt-4o-mini",
    rate_limit = 350,
    period = 60,
)

prompt = ChatCompletionPrompt([Message.from_dict(m) for m in prompt_text])
answer = await llm.acreate(prompt)

answer.content

If you want to use it with Ollama, make sure that Ollama is installed and running (on Linux/Mac, run `ollama serve` in the terminal). Also, ensure that the selected model is running—for example, by executing `ollama run gemma3:4b` in another terminal window.

In [ ]:
llm_ollama = LLMPromptProcessor(
    api_key='ollama',
    model="gemma3:4b",
    base_url = "http://localhost:11434/v1",
    rate_limit = 350,
    period = 60,
)

prompt = ChatCompletionPrompt([Message.from_dict(m) for m in prompt_text])
answer = await llm_ollama.acreate(prompt)

answer.content

## Connecting to Vector Databases (VDB)
The `VectorStore` class is currently limited in functionality—it supports only storing and retrieving embeddings. You can upsert items using the add function and retrieve similar items using the search function. Currently, it supports [FAISS](https://github.com/facebookresearch/faiss) and [USearch](https://github.com/unum-cloud/usearch) as vector databases.

The add function expects a list of `Embedding` objects, while the search function returns a list of `EmbeddingSimilarity` results, each of which includes an `Embedding` and its cosine similarity score.

The `Embedding` class holds the content (id), a unique ID (embedding_content), the embedding vector values (embedding_value), and optional metadata (metadata).

In [ ]:
from sziaaitls.core.embedders import Embedding
from sziaaitls.core.vector_stores import VectorStore
row1 = pdf_chunks.iloc[0].to_dict()
row2 = pdf_chunks.iloc[1].to_dict()
row3= pdf_chunks.iloc[2].to_dict()

embedding_list = [
    Embedding(id=0, embedding_content=row1['chunk_content'], embedding_value=row1['content_embedding'], 
              embedding_metadata={'uri':row1['uri'], 'chunk_order':row1['chunk_order']}), 
    Embedding(id=1, embedding_content=row2['chunk_content'], embedding_value=row2['content_embedding'], 
              embedding_metadata={'uri':row2['uri'], 'chunk_order':row2['chunk_order']}),
    Embedding(id=2, embedding_content=row3['chunk_content'], embedding_value=row3['content_embedding'], 
              embedding_metadata={'uri':row3['uri'], 'chunk_order':row3['chunk_order']}),
    ]

vector_store = VectorStore(name="usearch", dimension=1536)
await vector_store.add(embedding_list)

results = await vector_store.search(row2['content_embedding'], k=3)
[(res.embedding.id, res.similarity) for res in results]

## The SimpleRAG Class

The `SimpleRAG` class connects a `DocumentLibrary` and a `VectorStore` to create a minimal Retrieval-Augmented Generation (RAG) pipeline.

It allows you to:
- Build a vector index by embedding and storing document chunks.
- Perform similarity-based searches against this index using natural language queries.

The class supports both OpenAI and Ollama-compatible models for embedding and can use FAISS or USearch as the vector backend. The `create()` class method initializes and populates the index from a `DocumentLibrary`. The `search_query()` method retrieves relevant chunks with optional filtering by token limits, returning ranked results with cosine similarity scores.

In [ ]:
# testing RAG solutions
from sziaaitls.core.rag_solutions import SimpleRAG
from sziaaitls.core.vector_stores import VectorStore
from sziaaitls.core.embedders import TextEmbedder

# initializing a vector store and an embedder
vector_store = VectorStore(name="usearch", dimension=1536)
embedder = TextEmbedder("openai", api_key=os.environ.get("OPENAI_API_KEY"), model="text-embedding-3-small", rate_limit=3000, period=60, max_batch_tokens=32768)

# creating a SimpleRAG instance
simple_rag = await SimpleRAG.create(mydoclib, embedder, vector_store) # using the document library from above
print(f"RAG Store is created with {simple_rag.next_id -1} items.")


pd.DataFrame(simple_rag.metadata_list).head()

In [ ]:
findings = await simple_rag.search_query("What are transformers used for?", max_results=2)

for i in range(len(findings)):
    print(
        f"{findings[i].embedding.metadata['uri']}, \
        chunk: {findings[i].embedding.metadata['chunk_order']}, \
        similarity: {findings[i].similarity} \
        \ncontent: {findings[i].embedding.embedding_content}\n\n"
    )


**F. I. N.**